In [ ]:
class Tree:
    # nested position class
    class position:
        def element(self):
            raise NotImplementedError('must be implemented by subclass')

        def __eq__(self, other):
            raise NotImplementedError('must be implemented by subclass')

        def __ne__(self, other):
            return not (self == other)

    # abstract classes that concrete subclass must support
    def root(self):
        raise NotImplementedError('must be implemented by subclass')

    def parent(self):
        raise NotImplementedError('must be implemented by subclass')

    def num_childern(self):
        raise NotImplementedError('must be implemented by subclass')

    def childern(self):
        raise NotImplementedError('must be implemented by subclass')

    def __len__(self):
        raise NotImplementedError('must be implemented by subclass')

    def is_root(self):
        raise NotImplementedError('must be implemented by subclass')

    def is_leaf(self):
        raise NotImplementedError('must be implemented by subclass')

    def is_empty(self):
        raise NotImplementedError('must be implemented by subclass')

    def depth(self, p):
        if self.is_root(p):
            return 0
        else:
            return 1 + self.depth(self.parent(p))

    def _depth(self, p):
        return max(self.depth(p) for p in self.positions() if self.is_leaf(p))

    def _height2(self, p):
        if self.is_root(p):
            return 0
        else:
            return 1 + max(self._height2(c) for c in self.childern(p))

    def height(self, p):
        if p is None:
            p = self.root()
        return self._height2(p)


class BinaryTree(Tree):
    def left(self, p):
        raise NotImplementedError('must be implemented by subclass')

    def right(self, p):
        raise NotImplementedError('must be implemented by subclass')

    def sibling(self, p):
        parent = self.parent(p)
        if parent is None:
            return None
        else:
            if p == self.left(parent):
                return self.right(parent)
            else:
                return self.left(parent)

    def childern(self, p):
        if self.left(p) is not None:
            yield self.left(p)
        if self.right(p) is not None:
            yield self.right(p)


class LinkedBinaryTree(BinaryTree):
    class _Node:
        __slots__ = '_element', '_parent', '_left', '_right'

        def __init__(self, element, parent=None, left=None, right=None):
            self._element = element
            self._parent = parent
            self._left = left
            self._right = right

    class Position(BinaryTree.Position):
        def __int__(self, container, node):
            self._container = container
            self._node = node

        def element(self):
            return self._node._element

        def __eq__(self, other):
            return type(other) is type(self) and other._node is self._node

    def _validate(self, p):
        if not isinstance(p, self.Position):
            raise TypeError('p must be proper Position type')
        if p._container is not self:
            raise ValueError('p does not belong to this container')
        if p._node._parent is p._node:
            raise ValueError('p is no longer valid')
        return p._node

    def _make_position(self, node):
        return self.Position(self, node) if node is not None else None

    # -------------------------- binary tree constructor --------------------------
    def init(self):

        """Create an initially empty binary tree."""
        self.root = None
        self.size = 0

    # -------------------------- public accessors --------------------------
    def len(self):
        """Return the total number of elements in the tree."""

        return self.size

    def root(self):
        """Return the root Position of the tree (or None if tree is empty)."""

        return self._make_position(self.root)

    def parent(self, p):
        """Return the Position of p s parent (or None if p is root)."""

        node = self._validate(p)
        return self._make_position(node.parent)

    def left(self, p):
        """Return the Position of p s left child (or None if no left child)."""

        node = self._validate(p)
        return self._make_position(node.left)

    def right(self, p):
        """Return the Position of p s right child (or None if no right child)."""

        node = self._validate(p)
        return self._make_position(node.right)

    def num_children(self, p):

        """Return the number of children of Position p."""
        node = self._validate(p)
        count = 0
        # left child exists
        if node.left is not None:
            count += 1
        # right child exists
        if node.right is not None:
            count += 1
        return count

    def add_root(self, e):

        """Place element e at the root of an empty tree and return new Position.
        Raise ValueError if tree nonempty.
        """

    if self.root is not None:
        raise ValueError('Root exists')
    self.size = 1
    self.root = self._Node(e)
    return self.make_position(self.root)

    def add_left(self, p, e):
        """Create a new left child for Position p, storing element e.
        Return the Position of new node.
        Raise ValueError if Position p is invalid or p already has a left child.
        """
        node = self._validate(p)
        if node.left is not None:
            raise ValueError('Left child exists')
        self.size += 1
        # node is its parent
        node.left = self._Node(e, node)
        return self._make_position(node.left)

    def add_right(self, p, e):

        """Create a new right child for Position p, storing element e.
        Return the Position of new node.
        Raise ValueError if Position p is invalid or p already has a right child.
        """
        node = self._validate(p)
        if node.right is not None: raise ValueError('Right child exists')
        self.size += 1
        # node is its parent
        node.right = self._Node(e, node)
        return self._make_position(node.right)

    def replace(self, p, e):
        """Replace the element at position p with e, and return old element."""

        node = self._validate(p)
        old = node._element
        node._element = e
        return old

